# ds004350 (OpenNeuro) — SART Sustained Attention: All-Subjects Training + LDA / RF / SVM

This notebook will:
- Discover **all subjects/sessions** with SART files under `data/openneuro/ds004350_subset/`
- Load EEG from **EEGLAB `.set`** using `pymatreader` (avoids MNE EEGLAB event parsing issues)
- Read `*_events.tsv` and create **lapse labels** (commission errors on NO-GO)
- Preprocess EEG (avg ref, 1–40 Hz)
- Epoch trials and extract **bandpower features** (theta/alpha/beta per channel)
- Train and evaluate **LDA**, **Random Forest**, and **SVM** under:
  1) **Cross-session (within-subject):** train `ses-pre` → test `ses-post`
  2) **Leave-One-Subject-Out (LOSO):** group generalization across subjects

> Notes:
> - Lapse events are typically rare (heavy class imbalance). We report **balanced accuracy**, **ROC-AUC**, and **lapse recall/precision**.
> - If your `events.tsv` schema differs, adjust `label_sart_lapses()` once after inspecting a few files.

## Expected folder
`Neurotech_Model/data/openneuro/ds004350_subset/`


In [19]:
%pip install -U numpy pandas scipy scikit-learn mne pymatreader tqdm matplotlib pyarrow xgboost joblib

Note: you may need to restart the kernel to use updated packages.


In [21]:
# If needed (fresh env), uncomment and run:
# 

import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import mne
from pathlib import Path
from tqdm import tqdm
from scipy import signal

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score,
    balanced_accuracy_score,
    roc_auc_score,
    precision_recall_fscore_support
)
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GroupKFold

from pymatreader import read_mat

mne.set_log_level("WARNING")
print("MNE:", mne.__version__)


# Optional (for parquet cache + GPU boosting)
try:
    import pyarrow as pa
    print("pyarrow:", pa.__version__)
except Exception as e:
    print("pyarrow not available (parquet saving may fall back):", repr(e))

try:
    import xgboost as xgb
    print("xgboost:", xgb.__version__)
except Exception as e:
    print("xgboost not available (GPU boosting disabled):", repr(e))


MNE: 1.11.0
pyarrow: 23.0.0
xgboost: 3.1.3


In [22]:
# ---- Paths ----
# Run from Neurotech_Model repo root (so relative path works)
SART_ROOT = Path("../data/openneuro/ds004350_subset").resolve()
print("SART_ROOT:", SART_ROOT, "exists:", SART_ROOT.exists())

run_events = sorted(SART_ROOT.glob("sub-*/ses-*/eeg/*task-SART*_events.tsv"))
print("Found SART runs (events.tsv):", len(run_events))
for p in run_events[:8]:
    print(" -", p.relative_to(SART_ROOT))


SART_ROOT: /Users/tanmmay/Coding_projects/Neurotech_Model/data/openneuro/ds004350_subset exists: True
Found SART runs (events.tsv): 48
 - sub-001/ses-post/eeg/sub-001_ses-post_task-SART_events.tsv
 - sub-001/ses-pre/eeg/sub-001_ses-pre_task-SART_events.tsv
 - sub-002/ses-post/eeg/sub-002_ses-post_task-SART_events.tsv
 - sub-002/ses-pre/eeg/sub-002_ses-pre_task-SART_events.tsv
 - sub-003/ses-post/eeg/sub-003_ses-post_task-SART_events.tsv
 - sub-003/ses-pre/eeg/sub-003_ses-pre_task-SART_events.tsv
 - sub-004/ses-post/eeg/sub-004_ses-post_task-SART_events.tsv
 - sub-004/ses-pre/eeg/sub-004_ses-pre_task-SART_events.tsv


## 1) Helpers: parse IDs, label lapses, load EEG from `.set` (robust)

We load EEG data from `.set` via `pymatreader` and build an `mne.RawArray`.
We **do not** use EEGLAB events from the `.set` (we rely on `events.tsv`).


In [23]:
def parse_subject_session(events_path: Path) -> tuple[str, str]:
    sub = events_path.parts[-4]  # sub-001
    ses = events_path.parts[-3]  # ses-pre
    return sub, ses

def read_events_tsv(path: Path) -> pd.DataFrame:
    df = pd.read_csv(path, sep="\t")
    if "onset" not in df.columns:
        raise KeyError(f"'onset' missing in {path.name}. Columns={df.columns.tolist()}")
    return df

def label_sart_lapses(df_events: pd.DataFrame, no_go_digit: int = 3) -> pd.DataFrame:
    df = df_events.copy()
    cols = {c.lower(): c for c in df.columns}

    # ---------- 1) Identify stimulus digit / NO-GO ----------
    is_nogo = None

    # If trial_type exists, try to extract a digit or "nogo"
    if "trial_type" in cols:
        tt = df[cols["trial_type"]].astype(str).str.lower()
        digit = pd.to_numeric(tt.str.extract(r"([0-9])")[0], errors="coerce")
        is_nogo = (tt.str.contains("nogo") | tt.str.contains("no-go") | (digit == no_go_digit))
    elif "stimulus" in cols:
        stim = df[cols["stimulus"]].astype(str).str.lower()
        digit = pd.to_numeric(stim.str.extract(r"([0-9])")[0], errors="coerce")
        is_nogo = (stim.str.contains("nogo") | (digit == no_go_digit))
    elif "value" in cols:
        val = df[cols["value"]].astype(str).str.lower()
        digit = pd.to_numeric(val.str.extract(r"([0-9])")[0], errors="coerce")
        is_nogo = (val.str.contains("nogo") | (digit == no_go_digit))
    else:
        raise KeyError(f"Can't find trial_type/stimulus/value in events columns: {df.columns.tolist()}")

    # ---------- 2) Determine whether participant responded ----------
    rt_col = None
    for k in ["response_time", "reaction_time", "rt"]:
        if k in cols:
            rt_col = cols[k]
            break

    # Prefer explicit response column if present (more reliable than RT)
    resp_col = None
    for k in ["response", "key", "button", "pressed"]:
        if k in cols:
            resp_col = cols[k]
            break

    if resp_col is not None:
        s = df[resp_col]
        if pd.api.types.is_numeric_dtype(s):
            responded = s.notna() & (s != 0)
        else:
            ss = s.astype(str).str.lower()
            responded = ~ss.isin(["", "nan", "none", "n/a", "na", "0", "false"])
    elif rt_col is not None:
        rt = pd.to_numeric(df[rt_col], errors="coerce")
        responded = rt.notna()
    else:
        # If no response info exists, fall back to correctness if available
        responded = None

    # ---------- 3) If correct/accuracy exists, use it robustly ----------
    correct = None
    if "correct" in cols:
        s = df[cols["correct"]]
        if pd.api.types.is_numeric_dtype(s):
            correct = (s.fillna(1) != 0)
        else:
            ss = s.astype(str).str.lower()
            correct = ss.isin(["1", "true", "correct", "yes"])
    elif "accuracy" in cols:
        s = df[cols["accuracy"]]
        if pd.api.types.is_numeric_dtype(s):
            correct = (s.fillna(1) != 0)
        else:
            ss = s.astype(str).str.lower()
            correct = ss.isin(["1", "true", "correct", "yes"])

    # ---------- 4) Define lapses ----------
    # SART lapse = commission (respond on no-go) OR omission (no response on go)
    if responded is not None:
        commission = is_nogo & responded
        omission   = (~is_nogo) & (~responded)
        lapse = (commission | omission).astype(int)
    elif correct is not None:
        # fallback: any incorrect is a lapse
        lapse = (~correct).astype(int)
    else:
        raise KeyError(f"No response/RT and no correct/accuracy columns found: {df.columns.tolist()}")

    df["is_nogo"] = is_nogo.astype(int)
    df["y"] = lapse.astype(int)      # <-- use y consistently
    return df


def load_eeg_from_set(events_tsv: Path) -> mne.io.RawArray:
    base = events_tsv.name.replace("_events.tsv", "")
    set_path = events_tsv.with_name(base + "_eeg.set")
    ch_tsv   = events_tsv.with_name(base + "_channels.tsv")
    if not set_path.exists():
        raise FileNotFoundError(f"Missing {set_path}")

    mat = read_mat(str(set_path))
    EEG = mat.get("EEG", mat)

    data = np.array(EEG["data"])
    srate = float(EEG["srate"])

    # Ensure (n_ch, n_samp)
    nbchan = int(EEG.get("nbchan", data.shape[0]))
    if data.shape[0] != nbchan and data.shape[1] == nbchan:
        data = data.T

    # Channel names
    ch_names = None
    if ch_tsv.exists():
        chdf = pd.read_csv(ch_tsv, sep="\t")
        if "name" in chdf.columns and len(chdf["name"]) == data.shape[0]:
            ch_names = chdf["name"].astype(str).tolist()

    if ch_names is None:
        chanlocs = EEG.get("chanlocs", [])
        if isinstance(chanlocs, list) and len(chanlocs) == data.shape[0]:
            ch_names = [c.get("labels", f"ch{i}") for i, c in enumerate(chanlocs)]
        else:
            ch_names = [f"ch{i}" for i in range(data.shape[0])]

    # Scale heuristic: if values look like microvolts, convert to volts
    peak = float(np.nanmax(np.abs(data)))
    if peak > 1e3:
        data = data * 1e-6

    info = mne.create_info(ch_names=ch_names, sfreq=srate, ch_types=["eeg"] * len(ch_names))
    raw = mne.io.RawArray(data.astype(np.float64), info, verbose=False)
    return raw


## 2) Feature extraction: theta/alpha/beta bandpower per channel

Feature vector length = `3 * n_channels`.


In [24]:
BANDS = {
    "theta": (4, 8),
    "alpha": (8, 13),
    "beta":  (13, 30),
}

import numpy as np
from scipy.signal import welch

def bandpower_epoch(epoch, sf, fmin, fmax, nperseg=256):
    """
    epoch: (n_channels, n_times) OR (n_times,)
    returns: (n_channels,) bandpower
    """
    epoch = np.asarray(epoch)

    # Ensure shape is (n_channels, n_times)
    if epoch.ndim == 1:
        epoch = epoch[None, :]   # 1 channel
    elif epoch.ndim == 3:
        # if someone accidentally passes (n_epochs, n_channels, n_times),
        # this function should not accept it
        raise ValueError(f"bandpower_epoch expected 1D or 2D, got {epoch.shape}")

    nperseg = min(nperseg, epoch.shape[-1])

    freqs, psd = welch(epoch, fs=sf, axis=-1, nperseg=nperseg)  # psd: (n_channels, n_freqs)

    idx = (freqs >= fmin) & (freqs <= fmax)
    if not np.any(idx):
        # no freqs in band: return zeros instead of crashing
        return np.zeros(epoch.shape[0], dtype=np.float32)

    bp = np.trapezoid(psd[:, idx], freqs[idx], axis=1)  # (n_channels,)
    return bp.astype(np.float32)


def make_feature_matrix(epochs: mne.Epochs) -> np.ndarray:
    data = epochs.get_data()  # (n_epochs, n_ch, n_times)
    sf = epochs.info["sfreq"]
    feats = []
    for ep in data:
        parts = []
        for (f1, f2) in BANDS.values():
            parts.append(bandpower_epoch(ep, sf, f1, f2))
        feats.append(np.concatenate(parts, axis=0))
    return np.asarray(feats, dtype=np.float32)


## 3) Build dataset across all runs (and cache)

This step loads EEG, epochs trials, and computes features.
A cache is saved to speed up reruns.


In [25]:
DERIVED_DIR = Path("data/derived").resolve()
DERIVED_DIR.mkdir(parents=True, exist_ok=True)
CACHE_PATH = DERIVED_DIR / "ds004350_sart_features.parquet"

from pathlib import Path
import pandas as pd

def _safe_load_cache(cache_path):
    cache_path = Path(cache_path)

    # 1) Try parquet
    if cache_path.exists():
        try:
            df = pd.read_parquet(cache_path)
            print("✅ Loaded cache (parquet):", cache_path, "shape:", df.shape)
            return df
        except Exception as e:
            print("⚠️ Failed to read parquet cache:", repr(e))

    # 2) Try pickle fallback
    pkl_path = cache_path.with_suffix(".pkl")
    if pkl_path.exists():
        df = pd.read_pickle(pkl_path)
        print("✅ Loaded cache (pickle):", pkl_path, "shape:", df.shape)
        return df

    # 3) Nothing found
    return None


def _safe_save_cache(df: pd.DataFrame, cache_path: Path) -> Path:
    """Save parquet cache safely; if pyarrow registry issues occur, fall back to pickle."""
    cache_path.parent.mkdir(parents=True, exist_ok=True)

    # Ensure features are plain Python lists (avoids Arrow extension type issues)
    if "features" in df.columns:
        df = df.copy()
        df["features"] = df["features"].apply(lambda x: list(x) if not isinstance(x, list) else x)

    # 1) Try normal pandas parquet
    try:
        df.to_parquet(cache_path, index=False)
        return cache_path
    except Exception as e1:
        # 2) Try direct pyarrow write_table
        try:
            import pyarrow as pa
            import pyarrow.parquet as pq
            table = pa.Table.from_pandas(df, preserve_index=False)
            pq.write_table(table, str(cache_path))
            return cache_path
        except Exception as e2:
            # 3) Fallback: pickle
            pkl_path = cache_path.with_suffix(".pkl")
            df.to_pickle(pkl_path)
            print("Parquet save failed; saved pickle instead:", pkl_path, "| errors:", repr(e1), repr(e2))
            return pkl_path

def eval_predictions(y_true, y_pred, y_prob=None) -> dict:
    bal_acc = balanced_accuracy_score(y_true, y_pred)
    acc = accuracy_score(y_true, y_pred)
    pr, rc, f1, _ = precision_recall_fscore_support(
        y_true, y_pred, average=None, labels=[0, 1], zero_division=0
    )
    out = {
        "accuracy": acc,
        "balanced_accuracy": bal_acc,
        "precision_no_lapse": pr[0],
        "recall_no_lapse": rc[0],
        "f1_no_lapse": f1[0],
        "precision_lapse": pr[1],
        "recall_lapse": rc[1],
        "f1_lapse": f1[1],
    }
    if y_prob is not None and len(np.unique(y_true)) == 2:
        try:
            out["roc_auc"] = roc_auc_score(y_true, y_prob)
        except Exception:
            out["roc_auc"] = np.nan
    else:
        out["roc_auc"] = np.nan
    return out

def build_or_load_features(
    cache_path: Path = CACHE_PATH,
    tmin: float = 0.0,
    tmax: float = 1.5,
    l_freq: float = 1.0,
    h_freq: float = 40.0,
) -> pd.DataFrame:

    # ---------- Load cache safely ----------
    df_cached = _safe_load_cache(cache_path)
    if df_cached is not None and len(df_cached) > 0:
        print("Using cached features:", cache_path)
        return df_cached

    # ---------- Build ----------
    rows = []
    for ev_path in tqdm(run_events, desc="Runs"):
        sub, ses = parse_subject_session(ev_path)

        ev = read_events_tsv(ev_path)
        ev = label_sart_lapses(ev)  # produces ev["y"]

        # Load EEG
        raw = load_eeg_from_set(ev_path)
        raw.pick_types(eeg=True)

        # Reference/filter (speed: allow multi-core if available)
        raw.set_eeg_reference("average", verbose=False)
        raw.filter(l_freq, h_freq, n_jobs=-1, verbose=False)

        sfreq = float(raw.info["sfreq"])
        onsets = ev["onset"].astype(float).to_numpy()

        # IMPORTANT: use y (not lapse)
        y_all = ev["y"].astype(int).to_numpy()

        # Build MNE events array
        samples = np.round(onsets * sfreq).astype(int)
        samples = np.clip(samples, 0, raw.n_times - 1)

        events = np.column_stack([
            samples,
            np.zeros(len(samples), dtype=int),
            np.ones(len(samples), dtype=int),
        ])

        epochs = mne.Epochs(
            raw,
            events,
            event_id={"trial": 1},
            tmin=tmin,
            tmax=tmax,
            baseline=None,
            preload=True,
            reject_by_annotation=True,
            verbose=False,
        )

        # ---------- Align labels to kept epochs ----------
        # epochs.selection are indices into the original "events" array
        if len(epochs) == 0:
            continue

        kept = epochs.selection
        y = y_all[kept]

        # sanity
        if len(y) != len(epochs):
            raise RuntimeError(f"Label/epoch mismatch after selection: y={len(y)} epochs={len(epochs)}")

        X = make_feature_matrix(epochs)
        if X.shape[0] != len(y):
            raise RuntimeError(f"Feature/label mismatch: X={X.shape} y={len(y)}")

        for i in range(X.shape[0]):
            rows.append({
                "subject": sub,
                "session": ses,
                "trial_idx": int(i),
                "y": int(y[i]),
                "n_channels": int(epochs.info["nchan"]),
                "sfreq": float(epochs.info["sfreq"]),
                "features": X[i].tolist(),
            })

    df = pd.DataFrame(rows)

    if len(df) == 0:
        raise ValueError(
            "No rows were created. Check that run_events is non-empty, epochs are being created, "
            "and label_sart_lapses is producing y."
        )

    print("Built features:", df.shape, "overall lapse rate:", float(df["y"].mean()))
    saved_path = _safe_save_cache(df, cache_path)
    print("Saved cache:", saved_path)
    return df


df = build_or_load_features()
df.head()


✅ Loaded cache (parquet): /Users/tanmmay/Coding_projects/Neurotech_Model/notebooks/data/derived/ds004350_sart_features.parquet shape: (27704, 7)
Using cached features: /Users/tanmmay/Coding_projects/Neurotech_Model/notebooks/data/derived/ds004350_sart_features.parquet


,subject,session,trial_idx,y,n_channels,sfreq,features
0,sub-001,ses-post,0,0,64,256.0,"[4.147689570271895e-12, 5.5804479960619435e-12..."
1,sub-001,ses-post,1,0,64,256.0,"[5.5305612520206715e-12, 8.108990909305724e-12..."
2,sub-001,ses-post,2,0,64,256.0,"[4.2431071677062615e-12, 5.568594630550594e-12..."
3,sub-001,ses-post,3,0,64,256.0,"[2.105608439342821e-12, 2.199622211804253e-12,..."
4,sub-001,ses-post,4,0,64,256.0,"[3.6186600591014972e-12, 4.984446883016247e-12..."


## 4) Prepare matrices

In [26]:
import numpy as np

pos_total = 0
n_total = 0

for p in run_events[:10]:
    ev = read_events_tsv(p)
    ev = label_sart_lapses(ev)
    pos = int(ev["y"].sum()) if "y" in ev.columns else 0
    pos_total += pos
    n_total += len(ev)
    print(p.name, "pos:", pos, "/", len(ev))

print("Total positives across 10 runs:", pos_total, "/", n_total)


sub-001_ses-post_task-SART_events.tsv pos: 25 / 575
sub-001_ses-pre_task-SART_events.tsv pos: 40 / 560
sub-002_ses-post_task-SART_events.tsv pos: 19 / 581
sub-002_ses-pre_task-SART_events.tsv pos: 36 / 564
sub-003_ses-post_task-SART_events.tsv pos: 25 / 575
sub-003_ses-pre_task-SART_events.tsv pos: 19 / 581
sub-004_ses-post_task-SART_events.tsv pos: 38 / 544
sub-004_ses-pre_task-SART_events.tsv pos: 25 / 575
sub-005_ses-post_task-SART_events.tsv pos: 30 / 570
sub-005_ses-pre_task-SART_events.tsv pos: 32 / 568
Total positives across 10 runs: 289 / 5693


In [27]:
X = np.vstack(df["features"].apply(np.array).values)
y = df["y"].astype(int).values
groups = df["subject"].values

print("X:", X.shape, "y:", y.shape)
print("Overall lapse rate:", y.mean())
print("Subjects:", df["subject"].nunique(), "Sessions:", df["session"].unique())

per_sub = df.groupby("subject")["y"].agg(["count","sum","mean"]).sort_values("mean", ascending=False)
display(per_sub.head(10))


X: (27704, 192) y: (27704,)
Overall lapse rate: 0.04966791799018192
Subjects: 24 Sessions: ['ses-post' 'ses-pre']


,count,sum,mean
subject,,,
sub-024,1647,153,0.092896
sub-022,1132,68,0.060071
sub-014,1133,67,0.059135
sub-013,1134,66,0.058201
sub-001,1135,65,0.057269
sub-004,1119,63,0.056300
sub-010,1138,62,0.054482
sub-005,1138,62,0.054482
sub-008,1141,59,0.051709


## 5) Define models (LDA / RF / SVM)

In [28]:
models = {
    "LDA": Pipeline([
        ("scaler", StandardScaler()),
        ("clf", LinearDiscriminantAnalysis())
    ]),
    "RF": RandomForestClassifier(
        n_estimators=400,
        random_state=42,
        n_jobs=-1,
        class_weight="balanced_subsample",
        max_depth=None
    ),
    "SVM-RBF": Pipeline([
        ("scaler", StandardScaler()),
        ("clf", SVC(kernel="rbf", probability=True, class_weight="balanced", C=2.0, gamma="scale"))
    ]),
}


# Optional: GPU-accelerated XGBoost baseline (much faster than RF/SVM on large feature sets)
try:
    from xgboost import XGBClassifier

    def _xgb_use_gpu() -> bool:
        # Works on Colab; safe fallback on local machines
        try:
            import subprocess, shlex
            out = subprocess.check_output(shlex.split("nvidia-smi -L"), stderr=subprocess.STDOUT).decode()
            return "GPU" in out
        except Exception:
            return False

    use_gpu = _xgb_use_gpu()
    xgb_kwargs = dict(
        objective="binary:logistic",
        eval_metric="aucpr",
        n_estimators=800,
        learning_rate=0.03,
        max_depth=6,
        subsample=0.8,
        colsample_bytree=0.8,
        reg_lambda=1.0,
        min_child_weight=1.0,
        gamma=0.0,
        random_state=42,
    )
    if use_gpu:
        xgb_kwargs.update(dict(tree_method="gpu_hist", predictor="gpu_predictor", device="cuda"))
        print("🚀 XGBoost will use GPU")
    else:
        xgb_kwargs.update(dict(tree_method="hist"))
        print("🧠 XGBoost will use CPU (GPU not detected)")

    models["XGB"] = XGBClassifier(**xgb_kwargs)
except Exception as e:
    print("XGBoost not available; skipping XGB model:", repr(e))



🧠 XGBoost will use CPU (GPU not detected)


## 6) Leave-One-Subject-Out evaluation (subject generalization)

In [29]:
# FAST SUBJECT-WISE EVAL (quick results)
# - Uses GroupShuffleSplit (hold out whole subjects once) instead of full CV
# - Trains only a couple fast models first
# - Prints metrics immediately

import numpy as np
import pandas as pd
from sklearn.model_selection import GroupShuffleSplit
from sklearn.base import clone

# Build X, y, groups if you haven't already
# X = np.vstack(df["features"].values).astype(np.float32)
# y = df["y"].astype(int).to_numpy()
# groups = df["subject"].to_numpy()

# 1) One subject-wise split (fast)
gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
tr, te = next(gss.split(X, y, groups=groups))

Xtr, Xte = X[tr], X[te]
ytr, yte = y[tr], y[te]

print("Train trials:", len(tr), "Test trials:", len(te))
print("Train lapse rate:", float(ytr.mean()), "Test lapse rate:", float(yte.mean()))
print("Train subjects:", len(np.unique(groups[tr])), "Test subjects:", len(np.unique(groups[te])))

# 2) Pick only fast models for a quick sanity check
# (Adjust names if your dict uses different keys)
fast_model_names = [n for n in ["LDA", "LogReg", "SGD", "XGB"] if n in models]
if not fast_model_names:
    # fallback: just take first 2 models
    fast_model_names = list(models.keys())[:2]

quick_results = []
for name in fast_model_names:
    m = clone(models[name])
    m.fit(Xtr, ytr)

    yhat = m.predict(Xte)

    prob = None
    if hasattr(m, "predict_proba"):
        prob = m.predict_proba(Xte)[:, 1]
    elif hasattr(m, "decision_function"):
        scores = m.decision_function(Xte)
        prob = (scores - scores.min()) / (scores.max() - scores.min() + 1e-9)

    agg = eval_predictions(yte, yhat, prob)
    agg["model"] = name
    agg["n_test"] = int(len(yte))
    agg["test_lapse_rate"] = float(yte.mean())
    quick_results.append(agg)

res_fast = pd.DataFrame(quick_results).sort_values("balanced_accuracy", ascending=False)
display(res_fast)


Train trials: 22255 Test trials: 5449
Train lapse rate: 0.050640305549314764 Test lapse rate: 0.04569645806570013
Train subjects: 19 Test subjects: 5


,accuracy,balanced_accuracy,precision_no_lapse,recall_no_lapse,f1_no_lapse,precision_lapse,recall_lapse,f1_lapse,roc_auc,model,n_test,test_lapse_rate
1,0.954304,0.500000,0.954304,1.000000,0.976618,0.0,0.0,0.0,0.717149,XGB,5449,0.045696
0,0.953019,0.499327,0.954245,0.998654,0.975944,0.0,0.0,0.0,0.674992,LDA,5449,0.045696


## 7) Cross-session (within subject): train `ses-pre` → test `ses-post`

In [30]:
def cross_session_eval(df: pd.DataFrame, model) -> dict:
    subs = sorted(df["subject"].unique())
    all_true, all_pred, all_prob = [], [], []
    kept = 0

    for sub in subs:
        dsub = df[df["subject"] == sub]
        if not (("ses-pre" in set(dsub["session"])) and ("ses-post" in set(dsub["session"]))):
            continue

        pre = dsub[dsub["session"] == "ses-pre"]
        post = dsub[dsub["session"] == "ses-post"]

        Xpre = np.vstack(pre["features"].apply(np.array).values)
        ypre = pre["y"].astype(int).values
        Xpost = np.vstack(post["features"].apply(np.array).values)
        ypost = post["y"].astype(int).values

        # Need both classes in train/test for meaningful metrics
        if len(np.unique(ypre)) < 2 or len(np.unique(ypost)) < 2:
            continue

        model.fit(Xpre, ypre)
        yhat = model.predict(Xpost)

        prob = None
        if hasattr(model, "predict_proba"):
            prob = model.predict_proba(Xpost)[:, 1]
        elif hasattr(model, "decision_function"):
            scores = model.decision_function(Xpost)
            prob = (scores - scores.min()) / (scores.max() - scores.min() + 1e-9)

        all_true.append(ypost)
        all_pred.append(yhat)
        if prob is not None:
            all_prob.append(prob)

        kept += 1

    if kept == 0:
        return {"error": "No subjects had both sessions with lapses in both train and test."}

    y_true = np.concatenate(all_true)
    y_pred = np.concatenate(all_pred)
    y_prob = np.concatenate(all_prob) if len(all_prob) else None
    out = eval_predictions(y_true, y_pred, y_prob)
    out["n_subjects_used"] = int(kept)
    out["n_trials"] = int(len(y_true))
    out["lapse_rate"] = float(y_true.mean())
    return out

cross_rows = []
for name, model in models.items():
    out = cross_session_eval(df, model)
    out["model"] = name
    cross_rows.append(out)

cross_df = pd.DataFrame(cross_rows).sort_values("balanced_accuracy", ascending=False)
display(cross_df)


,accuracy,balanced_accuracy,precision_no_lapse,recall_no_lapse,f1_no_lapse,precision_lapse,recall_lapse,f1_lapse,roc_auc,n_subjects_used,n_trials,lapse_rate,model
0,0.766468,0.542361,0.954302,0.791865,0.865528,0.070157,0.292857,0.113197,0.582119,24,13754,0.050894,LDA
2,0.891813,0.527952,0.952009,0.933047,0.942433,0.089583,0.122857,0.103614,0.620672,24,13754,0.050894,SVM-RBF
3,0.947579,0.517447,0.950800,0.996323,0.973030,0.360000,0.038571,0.069677,0.670900,24,13754,0.050894,XGB
1,0.755853,0.491478,0.948059,0.785813,0.859345,0.047035,0.197143,0.075949,0.447126,24,13754,0.050894,RF


## 8) Optional: Inspect one subject’s confusion matrix quickly

Pick a subject and run pre→post for that subject only.


In [ ]:
# Save best-performing model (by balanced accuracy) after running the CV cell above
import joblib
from pathlib import Path

# res_df is created in the CV cell; pick best model name
best_name = res_df.iloc[0]["model"]
best_model = models[best_name]

# Fit on all data
best_model.fit(X, y)

OUT_DIR = Path("data/derived").resolve()
OUT_DIR.mkdir(parents=True, exist_ok=True)
model_path = OUT_DIR / f"best_model_{best_name}.joblib"
joblib.dump(best_model, model_path)
print("✅ Saved best model:", model_path)


In [33]:
from sklearn.metrics import ConfusionMatrixDisplay
import matplotlib.pyplot as plt

def subject_pre_post(df, subject: str, model):
    dsub = df[df["subject"] == subject]
    pre = dsub[dsub["session"] == "ses-pre"]
    post = dsub[dsub["session"] == "ses-post"]
    if pre.empty or post.empty:
        raise ValueError("Subject missing pre or post session.")

    Xpre = np.vstack(pre["features"].apply(np.array).values)
    ypre = pre["y"].astype(int).values
    Xpost = np.vstack(post["features"].apply(np.array).values)
    ypost = post["y"].astype(int).values

    model.fit(Xpre, ypre)
    yhat = model.predict(Xpost)

    print(f"{subject} pre→post balanced acc:", balanced_accuracy_score(ypost, yhat))
    cm = confusion_matrix(ypost, yhat, labels=[0,1])
    disp = ConfusionMatrixDisplay(cm, display_labels=["no-lapse","lapse"])
    disp.plot()
    plt.show()

# Example:
# subject_pre_post(df, "sub-001", models["SVM-RBF"])
